In [1]:
import pyfftw as ft
import numpy as np
from pyJHTDB import libJHTDB
import sys
import os 
import time
import math

#### Helper functions to Download data from JHTDB and to save them on the virtual machine (VM) in Scratch folder

In [2]:
def DownldData4Velocity(dataset_name,time,nx,ny,nz,auth_token):
    chkSz=32 #This is the maximum possible. May be increased in future depending on network bandwidth
    slabs=nx//chkSz
    lJHTDB=libJHTDB(auth_token)
    lJHTDB.initialize()#NOTE: datbase returns Velcocity as [nz,ny,nx,3]
    for k in range(slabs):
        start=np.array([k*chkSz,0,0],dtype=np.int)
        width=np.array([chkSz,ny,nz],dtype=np.int)
        uAll=lJHTDB.getRawData(time,start,width,data_set=dataset_name,getFunction='Velocity')
        if(k==0):
            vx=uAll[:,:,:,0]
            vy=uAll[:,:,:,1]
            vz=uAll[:,:,:,2]
        else:
            vx=np.concatenate((vx,uAll[:,:,:,0]),axis=2) #axis=2=> the index of nx
            vy=np.concatenate((vy,uAll[:,:,:,1]),axis=2)
            vz=np.concatenate((vz,uAll[:,:,:,2]),axis=2)
    lJHTDB.finalize()
    u=ft.zeros_aligned((nx,ny,nz),dtype='float32')
    v=ft.zeros_aligned((nx,ny,nz),dtype='float32')
    w=ft.zeros_aligned((nx,ny,nz),dtype='float32')
    u[:,:,:]=np.transpose(vx)
    v[:,:,:]=np.transpose(vy)
    w[:,:,:]=np.transpose(vz)
    return u,v,w

def DownldData4Pressure(dataset_name,time,nx,ny,nz,auth_token):
    chkSz=32 #This is the maximum possible. May be increased in future depending on network bandwidth
    slabs=nx//chkSz
    lJHTDB=libJHTDB(auth_token)
    lJHTDB.initialize()#NOTE: datbase returns Velcocity as [nz,ny,nx,3]
    for k in range(slabs):
        start=np.array([k*chkSz,0,0],dtype=np.int)
        width=np.array([chkSz,ny,nz],dtype=np.int)
        data=lJHTDB.getRawData(time,start,width,data_set=dataset_name,getFunction='Pressure')
        if(k==0):
            pTemp=data
        else:
            pTemp=np.concatenate((pTemp,data),axis=2) #axis=2=> the index of nx
    lJHTDB.finalize()
    p=ft.zeros_aligned((nx,ny,nz),dtype='float32')
    p[:,:,:]=np.transpose(pTemp)
    return p

def DownldData4Density(dataset_name,time,nx,ny,nz,auth_token):
    chkSz=32 #This is the maximum possible. May be increased in future depending on network bandwidth
    slabs=nx//chkSz
    lJHTDB=libJHTDB(auth_token)
    lJHTDB.initialize()#NOTE: datbase returns Velcocity as [nz,ny,nx,3]
    for k in range(slabs):
        start=np.array([k*chkSz,0,0],dtype=np.int)
        width=np.array([chkSz,ny,nz],dtype=np.int)
        data=lJHTDB.getRawData(time,start,width,data_set=dataset_name,getFunction='Density')
        if(k==0):
            dTemp=data
        else:
            dTemp=np.concatenate((dTemp,data),axis=2) #axis=2=> the index of nx
    lJHTDB.finalize()
    rho=ft.zeros_aligned((nx,ny,nz),dtype='float32')
    rho[:,:,:]=np.transpose(dTemp)
    return rho

def SaveVelDataOnVM(dirName,fileName,vx,vy,vz):
    outfile=dirName+fileName
    np.savez(outfile,vx=vx,vy=vy,vz=vz)
    return

def SavePressOnVM(dirName,fileName,p):
    outfile=dirName+fileName
    np.savez(outfile,p=p)
    return

def SaveDenOnVM(dirName,fileName,rho):
    outfile=dirName+fileName
    np.savez(outfile,rho=rho)
    return

# It loads velocity from the Scratch folder
def LoadVelDataFromVM(dirName,fileName,nx,ny,nz):
    outfile=dirName+fileName+'.npz'
    myfiles=np.load(outfile)
    vx=myfiles['vx']
    vy=myfiles['vy']
    vz=myfiles['vz']
    return vx,vy,vz

# It loads density from the Scratch folder
def LoadDenDataFromVM(dirName,fileName,nx,ny,nz):
    outfile=dirName+fileName+'.npz'
    myfiles=np.load(outfile)
    rho=myfiles['rho']
    return rho

# It loads Pressure from the Scratch folder
def LoadPresDataFromVM(dirName,fileName,nx,ny,nz):
    outfile=dirName+fileName+'.npz'
    myfiles=np.load(outfile)
    p=myfiles['p']
    return p

#### Downloads data from the Johns Hopkin Turbulence Database [replace authorization token with the one you have been provided]

In [3]:
# Use your own authorization token. Please visit "http://turbulence.pha.jhu.edu/help/authtoken.aspx" if you don't have one.
# edu.jhu.pha.turbulence.testing-201311
auth_token='edu.jhu.pha.turbulence.testing-201311'
dataset_name='mixing' # dataset_name='isotropic1024coarse'
t=11.4 # Snapshot time
N=1024 # Number of grids
nx=N
ny=N
nz=N

#### Saves the data on VM at "../mixing": Make sure that folder 'mixing' is present under the "scratch" directory.

In [4]:
t1=time.time()
rho=DownldData4Density(dataset_name,t,nx,ny,nz,auth_token)
t2=time.time()
sys.stdout.write('Done!! Cost= %0.2f seconds\n' %(t2-t1))

Done!! Cost= 1176.19 seconds


In [5]:
dirName='/home/idies/workspace/scratch/mixing/'
fileName='dendata_t11p4'
SaveDenOnVM(dirName,fileName,rho)
del rho # clears the memory
t2=time.time()
sys.stdout.write('Done!! Cost= %0.2f seconds\n' %(t2-t1))

Done!! Cost= 1305.50 seconds


In [6]:
t1=time.time()
p=DownldData4Pressure(dataset_name,t,nx,ny,nz,auth_token)
t2=time.time()
sys.stdout.write('Done!! Cost= %0.2f seconds\n' %(t2-t1))

Done!! Cost= 1194.42 seconds


In [7]:
dirName='/home/idies/workspace/scratch/mixing/'
fileName='presdata_t11p4'
t1=time.time()
SavePressOnVM(dirName,fileName,p)
del p # clears the memory
t2=time.time()
sys.stdout.write('Done!! Cost= %0.2f seconds\n' %(t2-t1))

Done!! Cost= 112.19 seconds


In [8]:
t1=time.time()
vx,vy,vz=DownldData4Velocity(dataset_name,t,nx,ny,nz,auth_token)
t2=time.time()
sys.stdout.write('Done!! Cost= %0.2f seconds\n' %(t2-t1))

Done!! Cost= 3824.47 seconds


In [9]:
dirName='/home/idies/workspace/scratch/mixing/'
fileName='veldata_t11p4'
t1=time.time()
SaveVelDataOnVM(dirName,fileName,vx,vy,vz)
del vx,vy,vz # clears the memory
t2=time.time()
sys.stdout.write('Done!! Cost= %0.2f seconds\n' %(t2-t1))

Done!! Cost= 284.43 seconds
